Anova_glove_50features

In [1]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [2]:
import os 
import numpy as np 
import pandas as pd 
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier 

In [3]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/ANOVA_Glove/data.csv"
df = pd.read_csv(path)
df.head() 

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,Level 5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,Level 5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,Level 5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",Level 5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Level 2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [4]:
df.iloc[:,1].replace(['Level 1', 'Level 2', 'Level 3', 'Level 4', 'Level 5'],
                        [1, 2, 3, 4, 5], inplace=True)

In [5]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [6]:
#define bins
bins = [0,2,4,5]
#name of the group 
group_names = ['Low', 'Moderate', 'High']
df['Level '] = pd.cut(df["Level "],bins,labels = group_names) 

In [7]:
df.head(5)

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,High,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,High,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,High,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",High,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Low,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [8]:
df['Level '] = df['Level '].map({'Low': 1, 'Moderate': 2, 'High': 3}).astype(int)

In [9]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,3,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,3,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,3,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",3,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,1,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [10]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/ANOVA_Glove/glove_withstopwordchangelevels_50_features.csv"
x = pd.read_csv(path)
x.head() 

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,40,41,42,43,44,45,46,47,48,49
0,0,-0.162205,-0.273445,0.088922,-1.158205,-0.682973,-0.522348,0.252270,0.450166,0.264311,...,0.359164,-0.296187,-0.959083,-0.284802,0.143591,0.590805,0.619011,-0.010133,-0.381799,-0.629083
1,1,-0.848897,0.856519,-0.489182,-0.408734,-0.552288,-0.133388,0.710629,0.711316,0.644359,...,0.347949,-0.049906,0.023303,-0.349564,1.137164,-0.037737,0.629554,0.027072,0.856284,-0.088800
2,2,-0.529911,0.973040,-0.136233,-0.693472,0.312765,-0.842224,0.240218,-0.647378,0.626205,...,0.553558,-0.065572,0.340320,-0.627935,1.029899,0.104229,-0.506119,0.415572,-0.011463,-0.262388
3,3,-0.774253,0.838091,-0.006255,0.736623,-0.366708,0.549797,0.753244,-0.467709,1.009183,...,1.759456,0.296629,-1.002400,-0.350040,-0.220311,-0.110786,0.395610,-0.495482,0.427047,0.482295
4,4,-0.537569,-0.236755,-0.325704,-0.583793,0.296968,0.652025,-0.962833,0.141703,0.415328,...,-0.515270,0.977412,0.125401,-0.526104,-1.131434,1.104740,-0.309647,-0.891839,0.698718,-0.364609


In [11]:
x.drop(x.columns[[0]], axis=1, inplace=True)
x.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,-0.162205,-0.273445,0.088922,-1.158205,-0.682973,-0.522348,0.252270,0.450166,0.264311,-0.184904,...,0.359164,-0.296187,-0.959083,-0.284802,0.143591,0.590805,0.619011,-0.010133,-0.381799,-0.629083
1,-0.848897,0.856519,-0.489182,-0.408734,-0.552288,-0.133388,0.710629,0.711316,0.644359,-0.363429,...,0.347949,-0.049906,0.023303,-0.349564,1.137164,-0.037737,0.629554,0.027072,0.856284,-0.088800
2,-0.529911,0.973040,-0.136233,-0.693472,0.312765,-0.842224,0.240218,-0.647378,0.626205,0.255603,...,0.553558,-0.065572,0.340320,-0.627935,1.029899,0.104229,-0.506119,0.415572,-0.011463,-0.262388
3,-0.774253,0.838091,-0.006255,0.736623,-0.366708,0.549797,0.753244,-0.467709,1.009183,-0.528502,...,1.759456,0.296629,-1.002400,-0.350040,-0.220311,-0.110786,0.395610,-0.495482,0.427047,0.482295
4,-0.537569,-0.236755,-0.325704,-0.583793,0.296968,0.652025,-0.962833,0.141703,0.415328,-0.257746,...,-0.515270,0.977412,0.125401,-0.526104,-1.131434,1.104740,-0.309647,-0.891839,0.698718,-0.364609


In [12]:
y=df['Level '] 

**Train Test Split**

In [13]:
# Train-Test-Split is performed

from sklearn.model_selection import train_test_split

# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=42)
X_train.shape, X_test.shape

((1220, 50), (524, 50))

In [14]:
print(y_train.value_counts(normalize=True)*100)
print("\n")
print(y_test.value_counts(normalize=True)*100)

2    45.327869
1    29.590164
3    25.081967
Name: Level , dtype: float64


2    45.229008
1    29.961832
3    24.809160
Name: Level , dtype: float64


**Model Building**

In [15]:
def perceptron_classifier (X_train, y_train, X_test, y_test):
  p = Perceptron(random_state=150)
  p.fit(X_train, y_train)
  predictions_train = p.predict(X_train)
  predictions_test = p.predict(X_test)
  train_score = accuracy_score(predictions_train, y_train)
  #print("score on train data: ", train_score)
  test_score = f1_score(predictions_test, y_test, average='weighted' )
  print("Perceptron F1 score:", test_score)

In [16]:
from sklearn.linear_model import Perceptron
perceptron_classifier(X_train, y_train, X_test, y_test) 

Perceptron F1 score: 0.3564997527121962


**Decision Tree**

In [18]:
clf_dt = DecisionTreeClassifier(max_depth=5, class_weight = "balanced")

In [19]:
clf_dt.fit(X_train,y_train)

DecisionTreeClassifier(class_weight='balanced', max_depth=5)

In [20]:
y_pred_train = clf_dt.predict(X_train)
y_pred_test = clf_dt.predict(X_test)

In [21]:
def classifcation_report_train_test(y_train, y_train_pred, y_test, y_test_pred):

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TRAIN DATA
            =========================================
            ''')
    print(classification_report(y_train, y_train_pred, digits=4))

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TEST DATA
            =========================================
            ''')
    print(classification_report(y_test, y_test_pred, digits=4))

In [22]:
scores = pd.DataFrame(columns=['Model', 'Train_Accuracy', 'Train_Recall', 'Train_Precision', 'Train_F1_Score', 
                               'Test_Accuracy', 'Test_Recall', 'Test_Precision', 'Test_F1_Score'])

def get_metrics(train_actual, train_predicted, test_actual, test_predicted, model_description, dataframe):

    train_accuracy  = accuracy_score(train_actual, train_predicted)
    train_recall    = recall_score(train_actual, train_predicted, average="weighted")
    train_precision = precision_score(train_actual, train_predicted, average="weighted")
    train_f1score   = f1_score(train_actual, train_predicted, average="weighted")
    
    test_accuracy   = accuracy_score(test_actual, test_predicted)
    test_recall     = recall_score(test_actual, test_predicted, average="weighted")
    test_precision  = precision_score(test_actual, test_predicted, average="weighted")
    test_f1score    = f1_score(test_actual, test_predicted, average="weighted")

    dataframe       = dataframe.append(pd.Series([model_description, 
                                                  train_accuracy, train_recall, train_precision, train_f1score,
                                                  test_accuracy, test_recall, test_precision, test_f1score],
                                                 index=scores.columns ), 
                                       ignore_index=True)

    return(dataframe)

In [23]:
scores = get_metrics(y_train, y_pred_train, y_test, y_pred_test, "DecisionTrees", scores)
scores

<ipython-input-22-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.444262,0.444262,0.511203,0.443482,0.307252,0.307252,0.34214,0.30718


**Random Forest**

In [24]:
clf_rf = RandomForestClassifier (class_weight = "balanced")

In [25]:
clf_rf.fit(X=X_train, y=y_train)

RandomForestClassifier(class_weight='balanced')

In [26]:
y_pred_train1 = clf_rf.predict(X_train)
y_pred_test1 = clf_rf.predict(X_test)

In [27]:
classifcation_report_train_test(y_train, y_pred_train1, y_test, y_pred_test1)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9830    0.9584    0.9705       361
           2     0.9712    0.9765    0.9739       553
           3     0.9487    0.9673    0.9579       306

    accuracy                         0.9689      1220
   macro avg     0.9676    0.9674    0.9674      1220
weighted avg     0.9690    0.9689    0.9689      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3099    0.1401    0.1930       157
           2     0.4344    0.7679    0.5549       237
           3     0.1176    0.0308    0.0488       130

    accuracy                         0.3969       524
   macro avg     0.2873    0.3129    0.2655       524
weighted avg     0.3185    0.3969    0.3209       524



In [28]:
scores = get_metrics(y_train, y_pred_train1, y_test, y_pred_test1, "RandomForest", scores)
scores

<ipython-input-22-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.444262,0.444262,0.511203,0.443482,0.307252,0.307252,0.342140,0.307180
1,RandomForest,0.968852,0.968852,0.969050,0.968879,0.396947,0.396947,0.318487,0.320889


**AdaBoost**

In [ ]:
clf_adaboost =  AdaBoostClassifier()

In [ ]:
clf_adaboost.fit(X_train, y_train)

AdaBoostClassifier()

In [ ]:
y_pred_train2 = clf_adaboost.predict(X_train)
y_pred_test2 = clf_adaboost.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train2, y_test, y_pred_test2)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.5869    0.3463    0.4355       361
           2     0.5285    0.7884    0.6328       553
           3     0.5385    0.3203    0.4016       306

    accuracy                         0.5402      1220
   macro avg     0.5513    0.4850    0.4900      1220
weighted avg     0.5483    0.5402    0.5165      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2727    0.1720    0.2109       157
           2     0.4646    0.6920    0.5559       237
           3     0.3194    0.1769    0.2277       130

    accuracy                         0.4084       524
   macro avg     0.3523    0.3470    0.3315       524
weighted avg     0.3711    0.4084    0.3711       524



In [ ]:
scores = get_metrics(y_train, y_pred_train2, y_test, y_pred_test2, "Adaboost", scores)
scores

<ipython-input-22-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.495902,0.495902,0.686965,0.378456,0.423664,0.423664,0.268520,0.288408
1,RandomForest,0.968852,0.968852,0.969000,0.968874,0.385496,0.385496,0.324232,0.324898
2,Adaboost,0.540164,0.540164,0.548259,0.516451,0.408397,0.408397,0.371095,0.371139


**Gradient Boosting Classifier**

In [ ]:
clf_gbm = GradientBoostingClassifier()

In [ ]:
clf_gbm.fit(X=X_train, y=y_train)

GradientBoostingClassifier()

In [ ]:
y_pred_train3 = clf_gbm.predict(X_train)
y_pred_test3 = clf_gbm.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train3, y_test, y_pred_test3)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9663    0.7950    0.8723       361
           2     0.7902    0.9873    0.8778       553
           3     0.9612    0.7288    0.8290       306

    accuracy                         0.8656      1220
   macro avg     0.9059    0.8370    0.8597      1220
weighted avg     0.8852    0.8656    0.8639      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3086    0.1592    0.2101       157
           2     0.4405    0.6878    0.5371       237
           3     0.2329    0.1308    0.1675       130

    accuracy                         0.3912       524
   macro avg     0.3274    0.3259    0.3049       524
weighted avg     0.3495    0.3912    0.3474       524



In [ ]:
scores = get_metrics(y_train, y_pred_train3, y_test, y_pred_test3, "GBM", scores)
scores

<ipython-input-22-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.495902,0.495902,0.686965,0.378456,0.423664,0.423664,0.268520,0.288408
1,RandomForest,0.968852,0.968852,0.969000,0.968874,0.385496,0.385496,0.324232,0.324898
2,Adaboost,0.540164,0.540164,0.548259,0.516451,0.408397,0.408397,0.371095,0.371139
3,GBM,0.865574,0.865574,0.885191,0.863950,0.391221,0.391221,0.349502,0.347408


**SVM Model**

In [29]:
linear_svm = SVC(kernel='linear', C=1, class_weight = "balanced")

In [30]:
linear_svm.fit(X=X_train, y= y_train)

SVC(C=1, class_weight='balanced', kernel='linear')

In [31]:
y_pred_train4 = linear_svm.predict(X_train)
y_pred_test4 = linear_svm.predict(X_test)

In [32]:
classifcation_report_train_test(y_train, y_pred_train4, y_test, y_pred_test4)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.4057    0.4349    0.4198       361
           2     0.5223    0.3382    0.4105       553
           3     0.3284    0.5098    0.3995       306

    accuracy                         0.4098      1220
   macro avg     0.4188    0.4276    0.4099      1220
weighted avg     0.4392    0.4098    0.4105      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2793    0.3185    0.2976       157
           2     0.4358    0.3291    0.3750       237
           3     0.2530    0.3231    0.2838       130

    accuracy                         0.3244       524
   macro avg     0.3227    0.3236    0.3188       524
weighted avg     0.3435    0.3244    0.3292       524



In [33]:
scores = get_metrics(y_train, y_pred_train4, y_test, y_pred_test4, "SVM", scores)
scores

<ipython-input-22-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.444262,0.444262,0.511203,0.443482,0.307252,0.307252,0.342140,0.307180
1,RandomForest,0.968852,0.968852,0.969050,0.968879,0.396947,0.396947,0.318487,0.320889
2,SVM,0.409836,0.409836,0.439186,0.410503,0.324427,0.324427,0.343550,0.329185


**KNN**

In [ ]:
knn = KNeighborsClassifier(n_neighbors=7)

In [ ]:
knn.fit(X=X_train, y=y_train)

KNeighborsClassifier(n_neighbors=7)

In [ ]:
y_pred_train5 = knn.predict(X_train)
y_pred_test5 = knn.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train5, y_test, y_pred_test5)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.5120    0.4737    0.4921       361
           2     0.5638    0.7595    0.6471       553
           3     0.5461    0.2516    0.3445       306

    accuracy                         0.5475      1220
   macro avg     0.5406    0.4949    0.4946      1220
weighted avg     0.5440    0.5475    0.5254      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3448    0.3185    0.3311       157
           2     0.4574    0.6118    0.5235       237
           3     0.2097    0.1000    0.1354       130

    accuracy                         0.3969       524
   macro avg     0.3373    0.3434    0.3300       524
weighted avg     0.3622    0.3969    0.3696       524



In [ ]:
scores = get_metrics(y_train, y_pred_train5, y_test, y_pred_test5, "KNN", scores)
scores

<ipython-input-22-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.495902,0.495902,0.686965,0.378456,0.423664,0.423664,0.268520,0.288408
1,RandomForest,0.968852,0.968852,0.969000,0.968874,0.385496,0.385496,0.324232,0.324898
2,Adaboost,0.540164,0.540164,0.548259,0.516451,0.408397,0.408397,0.371095,0.371139
3,GBM,0.865574,0.865574,0.885191,0.863950,0.391221,0.391221,0.349502,0.347408
4,SVM,0.472951,0.472951,0.367231,0.338314,0.442748,0.442748,0.289269,0.305245
5,KNN,0.547541,0.547541,0.544007,0.525360,0.396947,0.396947,0.362219,0.369565


**GNB**

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()

In [ ]:
nb.fit(X=X_train, y=y_train)

GaussianNB()

In [ ]:
y_pred_train6 = nb.predict(X_train)
y_pred_test6 = nb.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train6, y_test, y_pred_test6)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.3932    0.2244    0.2857       361
           2     0.4930    0.6401    0.5570       553
           3     0.3446    0.3333    0.3389       306

    accuracy                         0.4402      1220
   macro avg     0.4103    0.3993    0.3939      1220
weighted avg     0.4263    0.4402    0.4220      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3043    0.1338    0.1858       157
           2     0.4530    0.6709    0.5408       237
           3     0.2596    0.2077    0.2308       130

    accuracy                         0.3950       524
   macro avg     0.3390    0.3374    0.3191       524
weighted avg     0.3605    0.3950    0.3575       524



In [ ]:
scores = get_metrics(y_train, y_pred_train6, y_test, y_pred_test6, "GNB", scores)
scores

<ipython-input-22-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.495902,0.495902,0.686965,0.378456,0.423664,0.423664,0.268520,0.288408
1,RandomForest,0.968852,0.968852,0.969000,0.968874,0.385496,0.385496,0.324232,0.324898
2,Adaboost,0.540164,0.540164,0.548259,0.516451,0.408397,0.408397,0.371095,0.371139
3,GBM,0.865574,0.865574,0.885191,0.863950,0.391221,0.391221,0.349502,0.347408
4,SVM,0.472951,0.472951,0.367231,0.338314,0.442748,0.442748,0.289269,0.305245
5,KNN,0.547541,0.547541,0.544007,0.525360,0.396947,0.396947,0.362219,0.369565
6,GNB,0.440164,0.440164,0.426264,0.422034,0.395038,0.395038,0.360480,0.357539
